# Transactional Data from Whales- BigQuery

In [ ]:
# Import Libraries
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.cloud import bigquery

In [ ]:
# Define Google Cloud Credential.
# PLEASE BE AWARE THAT CREDENTIALS VARY DEPENDING ON EACH ACCOUNT.
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:/Users/JCRG/Desktop/Master/Dissertation/Code/Google Cloud/dissertation-352916-492433041777.json"
# Call Big Query API
bqclient = bigquery.Client()

In [ ]:
# Execute Query to get top balances in Ethereum
# Please be aware that temp table 'A' is created using the top 10,000 balances in Ethereum.
# Table temp 'A' needs to be uploaded to Google Cloud BigQuery in the account whose credentials were used before.
# The code can be found in the file 'Etherscan- Name Tags Known Addresses.ipynb'
query_string = """
WITH
A AS (SELECT * FROM `dissertation-352916.Data.top_100_address` LIMIT 10000),


B AS (SELECT * FROM A
JOIN `bigquery-public-data.crypto_ethereum.transactions` AS t ON (A.Address=t.from_address)
WHERE t.block_timestamp< '2022-06-30 00:00:00')

SELECT B.*,value*power(10, -18) AS value_ether FROM B
"""

dataframe = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(
        create_bqstorage_client=True,
    )
)

In [ ]:
# Export to dataframe
dataframe=pd.DataFrame(dataframe)
# Export to csv
dataframe.to_csv('Etherscan_Top10000_Addresses')